# **Линейная регрессия**

## **<span style='color:#F1A424'>Матричных Операции/Градиентный Спуск</span>**

Продолжаем использовать датасет Бостона, в котором нам нужно предсказывать стоимость жилья

### **Задание**

#### **Задание 3.6.1**

Реализуйте матричную линейную регрессию. Какой получился RMSE?


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [28]:
import torch
from torch.linalg import inv
from torch import matmul
from torch import nn
from torch.nn import MSELoss

y = df['medv']
X = df.drop(['medv'],axis=1)

# Добавляем вектор из единиц к нашим данным
X = np.hstack([np.ones(X.shape[0])[:,None], X])

# Преобразуем в тензора
X = torch.tensor(X)
y = torch.tensor(y.values)

# функция для вычисления параметров модели
def linreg_linear(X,y):
    lsm = inv(matmul(X.transpose(1,0),X))
    Xt = matmul(X.transpose(1,0),y.double())
    theta = matmul(lsm,Xt)
    return theta

# Получаем параметры модели
theta = linreg_linear(X,y)

# Предсказание модели на данных
y_pred = matmul(X,theta)

# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)


21.8948311817292 4.679191402553351


#### **Задание 3.6.2**

Постройте модель при помощи sklearn. Используйте параметры по умолчанию, обучите на всей выборке и посчитайте RMSE.

In [17]:
from sklearn.linear_model import LinearRegression as LR

df = pd.read_csv('data.csv')
y = df['medv']
X = df.drop(['medv'],axis=1)
X = np.hstack([np.ones(X.shape[0])[:,None], X])

model = LR()
model.fit(X,y)
y_pred = model.predict(X)

y = torch.tensor(y)
y_pred = torch.tensor(y_pred)

print_regression_metrics(y, y_pred)

21.894831181729202 4.679191402553352


#### **Задание 3.6.3**

У какого из признаков наибольшее стандартное отклонение? Чему оно равно?

In [25]:
pd.DataFrame(X.std(axis=0),index=list(df.columns),columns=['std val']).sort_values(by='std val',ascending=False)[:5].T

,ptratio,lstat,dis,indus,tax
std val,168.370495,91.204607,28.121033,23.299396,8.698651


#### **Задание 3.6.4**

Обучите регрессию без дополнительного столбца единиц. Какой получился RMSE?

In [31]:
import torch
from torch.linalg import inv
from torch import matmul
from torch import nn
from torch.nn import MSELoss

y = df['medv']
X = df.drop(['medv'],axis=1)

# Преобразуем в тензора
X = torch.tensor(X.values)
y = torch.tensor(y.values)

# функция для вычисления параметров модели
def linreg_linear(X,y):
    lsm = inv(matmul(X.transpose(1,0),X))
    Xt = matmul(X.transpose(1,0),y.double())
    theta = matmul(lsm,Xt)
    return theta

# Получаем параметры модели
theta = linreg_linear(X,y)

# Предсказание модели на данных
y_pred = matmul(X,theta)

# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)


24.166099330126492 4.915902799092604


#### **Задание 3.6.5**

Очистите данные от строк, где значение признака B меньше 50. Какой получился RMSE?

In [47]:
import torch
from torch.linalg import inv
from torch import matmul
from torch import nn
from torch.nn import MSELoss

df = pd.read_csv('data.csv')
ldf = df[~(df['black'] < 50)]
df = ldf

y = df['medv']
X = df.drop(['medv'],axis=1)

# Преобразуем в тензора
X = torch.tensor(X.values)
y = torch.tensor(y.values)

# функция для вычисления параметров модели
def linreg_linear(X,y):
    lsm = inv(matmul(X.transpose(1,0),X))
    Xt = matmul(X.transpose(1,0),y.double())
    theta = matmul(lsm,Xt)
    return theta

# Получаем параметры модели
theta = linreg_linear(X,y)

# Предсказание модели на данных
y_pred = matmul(X,theta)

# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)

23.784379755512255 4.876923287843705


#### **Задание 3.6.6**

Нормализуйте признаки и обучите линейную регрессию матричным методом. Какой получился RMSE?

In [44]:
import torch
from torch.linalg import inv
from torch import matmul
from torch import nn
from torch.nn import MSELoss

y = df['medv']
X = df.drop(['medv'],axis=1)

# Нормализуем данные
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

# Преобразуем в тензора
X = torch.tensor(X)
y = torch.tensor(y.values)

# функция для вычисления параметров модели
def linreg_linear(X,y):
    lsm = inv(matmul(X.transpose(1,0),X))
    Xt = matmul(X.transpose(1,0),y.double())
    theta = matmul(lsm,Xt)
    return theta

# Получаем параметры модели
theta = linreg_linear(X,y)

# Предсказание модели на данных
y_pred = matmul(X,theta)

# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)


21.894831181729202 4.679191402553352
